# Required Librarires

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
import numpy as np
import keras
from glob import glob
import matplotlib.pyplot as plt
import tensorflow as tf
tf.__version__

'2.9.2'

# Data Augmentation

In [ ]:
IMAGE_SIZE = [224, 224]
train_path = '/content/drive/MyDrive/Fundus Dataset/train'
valid_path = '/content/drive/MyDrive/Fundus Dataset/train'

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Fundus Dataset/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 2820 images belonging to 3 classes.


In [ ]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Fundus Dataset/valid',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 705 images belonging to 3 classes.


# Methodology

In [ ]:
vgg16 = VGG16(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)

In [ ]:
for layer in vgg16.layers:
  layer.trainable = False

In [ ]:
folders = glob('/content/drive/MyDrive/Fundus Dataset/train/*')
folders

['/content/drive/MyDrive/Fundus Dataset/train/diabetes',
 '/content/drive/MyDrive/Fundus Dataset/train/glaucoma',
 '/content/drive/MyDrive/Fundus Dataset/train/normal']

In [ ]:
x = Flatten()(vgg16.output)

In [ ]:
prediction = Dense(len(folders), activation = 'softmax')(x)
model = Model(inputs = vgg16.input, outputs = prediction)

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
model.compile(optimizer = 'sgd',
              loss = 'categorical_crossentropy',
              metrics = [keras.metrics.Precision(name = 'precision'),
                         keras.metrics.Recall(name = 'Recall'),
                         keras.metrics.SpecificityAtSensitivity(0.5, name = 'specificity'),
                         keras.metrics.SensitivityAtSpecificity(0.5, name = 'sensicivity'),
                         'accuracy'])

In [ ]:
r = model.fit(
    training_set,
    validation_data = test_set,
    epochs = 30,
    steps_per_epoch = len(training_set)
)

Epoch 1/30
89/89 [==============================] - 2343s 26s/step - loss: 1.0792 - precision: 0.5384 - Recall: 0.4326 - specificity: 0.7723 - sensicivity: 0.7975 - accuracy: 0.5053 - val_loss: 1.3549 - val_precision: 0.4057 - val_Recall: 0.3844 - val_specificity: 0.6255 - val_sensicivity: 0.6567 - val_accuracy: 0.4085
Epoch 2/30
89/89 [==============================] - 361s 4s/step - loss: 0.8469 - precision: 0.6387 - Recall: 0.5472 - specificity: 0.8684 - sensicivity: 0.8855 - accuracy: 0.6046 - val_loss: 1.0918 - val_precision: 0.5731 - val_Recall: 0.4723 - val_specificity: 0.8035 - val_sensicivity: 0.7348 - val_accuracy: 0.5291
Epoch 3/30
89/89 [==============================] - 358s 4s/step - loss: 0.8534 - precision: 0.6173 - Recall: 0.5496 - specificity: 0.8596 - sensicivity: 0.9067 - accuracy: 0.5947 - val_loss: 1.2202 - val_precision: 0.4591 - val_Recall: 0.4298 - val_specificity: 0.7064 - val_sensicivity: 0.8043 - val_accuracy: 0.4553
Epoch 4/30
89/89 [=======================

# Evaluation